In [ ]:
# %%
# import tensorflow as tf

# from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import skimage.measure

import random
import copy
import math
import cv2
import os

from IPython.display import clear_output

# %%
idx = 12
# if True:
for idx in range(13):
  
  div = 32
  step = int(32/4)
  #      0  1  2  3  4  5  6  7  8  9 10 11 12
  red = [6, 6, 5, 5, 2, 6, 6, 4, 6, 4, 4, 4, 6] # Check the estimated_scales.yaml -> the resolution has to be 0.4 meters/pixel
  red = red[idx]
  location_list = ['coupa', 'coupa', 'coupa', 'coupa', 'gates', 'hyang', 'hyang', 'hyang', 'hyang', 'little', 'nexus', 'nexus', 'deathCircle']
  number_list = ['0', '1', '2', '3', '2', '2', '3', '4', '10', '3', '0', '1', '0']

  # semantic class cash is used also for bike parking spots
  sem_dict = ['cash', 'entrance', 'light', 'sit', 'stairs', 'trash', 'tree','restricted','grass','intersection','shadow']
  chans = len(sem_dict)+1

  lut_in = [0, 20, 50, 100, 150, 255]
  lut_out = [0, 100, 180, 220, 240, 255]
  lut_8u = np.interp(np.arange(0, 256), lut_in, lut_out).astype(np.uint8)

  location = location_list[idx]
  number = number_list[idx]
  map_name = 'stanford_'+location+number
  directory_dataset = '/'+location+'/video'+number+'/'
  print(map_name)

  # %%
  # Code to check if the red variable is correct to scale to the correct homography. Check on google maps and note that each pixel should be 0.4 meters (32 pixels = 12.8m)

  cap = cv2.VideoCapture('Stanford_Drone_Dataset/video'+directory_dataset+'video.mp4')
  df = pd.read_csv('Stanford_Drone_Dataset/annotations'+directory_dataset+'annotations.txt', delimiter=' ', header=None)
  
  # Check if camera opened successfully
  if (cap.isOpened()== False): 
    print("Error opening video stream or file")
  ret, frame = cap.read()
  print(frame.shape)

  frame = skimage.measure.block_reduce(frame, (red,red,1), np.max)

  # plt.figure(figsize=(3,3))
  # im = plt.imshow(frame)
  # ax = plt.gca()
  # # ax.set_xticks(np.arange(0, 170, 5))
  # # ax.set_yticks(np.arange(0, 90, 5))
  # plt.show()

  # %%
  cap = cv2.VideoCapture('Stanford_Drone_Dataset/video'+directory_dataset+'video.mp4')
  df = pd.read_csv('Stanford_Drone_Dataset/annotations'+directory_dataset+'annotations.txt', delimiter=' ', header=None)
  
  # Check if camera opened successfully
  if (cap.isOpened()== False): 
    print("Error opening video stream or file")
  ret, frame = cap.read()
  cv2.imwrite('maps/semantics/'+map_name+'/reference.png', frame)
  cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
  ref_frame = frame

  h = frame.shape[0]
  w = frame.shape[1]

  dt = 1/cap.get(cv2.CAP_PROP_FPS)
  dt = 1.2
  # When everything done, release the video capture object
  cap.release()

  data = np.zeros((h,w))
  data_vel = np.zeros((h,w))
  data_stops = np.zeros((h,w))
  pos = {'x': 0.0, 'y': 0.0}
  vel = {'x': 0.0, 'y': 0.0}

  ratioy = (df[4].max())/h
  ratiox = (df[3].max())/w

  ped = df[df[9]=='Pedestrian']# filter only people
  ped = ped.reset_index()

  alpha = 0.2
  for trackid in ped[0].unique():
    ce = ped[ped[0]==trackid]
    ce = ce.reset_index()

    data_temp = np.zeros((h,w))
    vel_temp = np.zeros((h,w))
    stop_temp = np.zeros((h,w))
    for index, el in ce.iterrows():
      centerOfCircle = (int((el[2]+el[4])/2/ratioy),int((el[1]+el[3])/2/ratiox))
      if index==0:
        pos = {'x': centerOfCircle[0],'y': centerOfCircle[1]}
      vel = {'x': alpha*(centerOfCircle[0]-pos['x'])/dt/red + (1-alpha)*vel['x'],'y': alpha*(centerOfCircle[1]-pos['y'])/dt/red + (1-alpha)*vel['y']}
      pos = {'x': centerOfCircle[0],'y': centerOfCircle[1]}

      if centerOfCircle[0] < h and centerOfCircle[1] < w and (el[6] == 0 or el[7] == 1):
        data_temp[centerOfCircle] = 1
        if index > 0:
          vel_temp[centerOfCircle] = math.sqrt(vel['x']**2+vel['y']**2)
          if(vel_temp[centerOfCircle] < 1e-30):
            stop_temp[centerOfCircle] = 1

    if np.max(vel_temp) < 2:
      data_vel = data_vel * (vel_temp == 0) + data_vel * (vel_temp > 0)*0.5 + vel_temp * (data_vel > 0)*0.5 + vel_temp * (data_vel == 0)
      data = data + data_temp
      data_stops = data_stops + stop_temp
    # else:
    #   print('VEL EXEEDS 2')

  # data = np.clip(data,0,np.max(data)*0.2)/(np.max(data)*0.2)
  # data = cv2.LUT((data/np.max(data)*255).astype(np.uint8), lut_8u).astype(float)/255
  data = skimage.measure.block_reduce(data, (red,red), np.max)
  print(f'data max: {np.max(data)}')
  print(f'data min: {np.min(data)}')
  data_show = data/np.max(data)
  ref_frame_red = skimage.measure.block_reduce(ref_frame, (red,red,1), np.max)/510+0.5
  # data = np.subtract(data, np.full((data.shape[0], data.shape[1]), np.min(data)))/(np.max(data)-np.min(data))

  plt.figure(figsize=(6,5))
  # plt.imshow(np.multiply(np.stack((map_var[:,:,0],map_var[:,:,0],map_var[:,:,0]),axis=2), np.stack((np.full(data.shape,1),1-data,1-data),axis=2)))
  plt.imshow(np.multiply(ref_frame_red, np.stack((np.full(data_show.shape,1),1-data_show,1-data_show),axis=2)))
  plt.show()

  np.savetxt('maps/semantics/'+map_name+'/humandensity-'+map_name+'-new.csv', data*255, delimiter=',', fmt='%3d')

  # -----------------------------------------------------------------------------------

  data_vel = skimage.measure.block_reduce(data_vel, (red,red), np.max)
  print(f'data_vel max: {np.max(data_vel)}')
  print(f'data_vel min: {np.min(data_vel)}')
  data_vel_show = data_vel/np.max(data_vel)
  # data_vel = np.subtract(data_vel, np.full((data.shape[0], data.shape[1]), np.min(data_vel)))/(np.max(data_vel)-np.min(data_vel))

  plt.figure(figsize=(6,5))
  # plt.imshow(np.multiply(np.stack((map_var[:,:,0],map_var[:,:,0],map_var[:,:,0]),axis=2), np.stack((np.full(data_vel.shape,1),1-data_vel,1-data_vel),axis=2)))
  plt.imshow(np.multiply(ref_frame_red, np.stack((np.full(data_vel_show.shape,1),1-data_vel_show,1-data_vel_show),axis=2)))
  plt.show()

  np.savetxt('maps/semantics/'+map_name+'/humandensity-'+map_name+'-vel.csv', data_vel*255, delimiter=',', fmt='%3d')

  # -----------------------------------------------------------------------------------

  data_stops = skimage.measure.block_reduce(data_stops, (red,red), np.max)
  print(f'data_stops max: {np.max(data_stops)}')
  print(f'data_stops min: {np.min(data_stops)}')
  data_stops_show = data_stops/np.max(data_stops)
  # data_stops = np.subtract(data_stops, np.full((data.shape[0], data.shape[1]), np.min(data_stops)))/(np.max(data_stops)-np.min(data_stops))

  plt.figure(figsize=(6,5))
  # plt.imshow(np.multiply(np.stack((map_var[:,:,0],map_var[:,:,0],map_var[:,:,0]),axis=2), np.stack((np.full(data_stops.shape,1),1-data_stops,1-data_stops),axis=2)))
  plt.imshow(np.multiply(ref_frame_red, np.stack((np.full(data_stops_show.shape,1),1-data_stops_show,1-data_stops_show),axis=2)))
  plt.show()

  np.savetxt('maps/semantics/'+map_name+'/humandensity-'+map_name+'-stop.csv', data_stops*255, delimiter=',', fmt='%3d')


# PGM to CSV

  print(f'map: {map_name} --------------')
  direc = 'maps/semantics/'+map_name+'/'
  file_list = os.listdir(direc)
  i = 0
  while i < len(file_list):
      if not ".pgm" in file_list[i]:
          file_list.remove(file_list[i])
          i = i-1
      i = i+1

  for file in file_list:
      lines = 0
      print(direc+file)
      with open(direc+file) as f:
          lines = f.readlines()

      sizes = lines[2].split(' ')
      h = int(sizes[1])
      w = int(sizes[0])

      map = np.zeros((h,w))

      count = 0
      # Converts data to a list of integers
      for line in lines[3:-1]:
          map[int(count/w),count%w] = int(line)
          # map[int(count/w),count%w] = int(int(line)>254)*255
          count = count +1

      df = pd.DataFrame(map)
      df = df.astype(int)
      output_file = list(direc+file)
      output_file[-3:] = 'csv'
      output_file = ''.join(output_file)
      df.to_csv(output_file,header=None, index=None)

  # %%
  print(map_name)

  lines = 0
  with open('maps/semantics/'+map_name+'/'+map_name+'.csv') as f:
    lines = f.readlines()

  h = len(lines)
  w = len(lines[0].split(','))

  # Converts data to a list of integers
  map_var = []
  for line in lines:
    map_var.extend([int(c) for c in line.split(',')])

  for lab_class in sem_dict:
    lines = 0
    try:
      with open('maps/semantics/'+map_name+'/'+map_name+'_sem_'+lab_class+'.csv') as f:
        lines = f.readlines()

      hh = len(lines)
      ww = len(lines[0].split(','))

      if hh != h or ww != w:
        print(f'h: {h}\tw: {w}')
        print(f'h: {hh}\tw: {ww}')
        raise SystemExit("ERROR: Different sizes!!")

      # Converts data to a list of integers
      for line in lines:
        map_var.extend([int(c) for c in line.split(',')])

    except FileNotFoundError:
      for i in range(h):
        for j in range(w):
          map_var.extend([255])

  map_var = np.reshape(map_var,[chans,h,w])
  map_var = np.moveaxis(map_var, 0, -1)
  map_var = map_var/255

  print(map_var.shape)

  map_aux = map_var
  map_var = np.zeros((int(math.ceil(h/red)),int(math.ceil(w/red)),chans))

  for idx in range(chans):
    map_var[:,:,idx] = skimage.measure.block_reduce(map_aux[:,:,idx], (red,red), np.min)
  h, w, _ = map_var.shape

  print(map_var.shape)

  np.savetxt('maps/semantics/'+map_name+'/'+map_name+'-reduced.csv', map_var[:,:,0]*255, delimiter=',', fmt='%3d')
  for index, lab_class in enumerate(sem_dict):
    np.savetxt('maps/semantics/'+map_name+'/'+map_name+'_sem_'+lab_class+'-reduced.csv', map_var[:,:,index+1]*255, delimiter=',', fmt='%3d')

  print(sem_dict)
  plt.figure(figsize=(20,5))
  for i in range(len(sem_dict)):
    ax = plt.subplot(1, len(sem_dict), i+1)
    alp = 0.5
    ax.imshow(np.multiply(np.stack((map_var[:,:,0],map_var[:,:,0],map_var[:,:,0]),axis=2),np.stack((map_var[:,:,i+1],map_var[:,:,i+1],map_var[:,:,i+1]),axis=2)*alp+(1-alp)), vmin=0, vmax=1)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
  plt.show()